<div style="display:block">
    <div style="width: 20%; display: inline-block; text-align: left;">
        <img src="http://upload.wikimedia.org/wikipedia/en/0/0c/Mu_Sigma_Logo.jpg" style="height:75px; margin-left:0px" />
    </div>
    <div style="width: 59%; display: inline-block">
        <h1  style="text-align: center"> SMA - Negation and Drug Relation </h1><br>
        <div style="width: 100%; text-align: center; display: inline-block;"><i>Author:</i> <strong>Sruthi S, Varun Behl</strong> </div>
    </div>
    <div style="width: 20%; text-align: right; display: inline-block;">
        <div style="width: 100%; text-align: left; display: inline-block;">
            <i>Created: </i>
            <time datetime="2018-02-14" pubdate>13th Feb 2018</time>
            <i>Modified: </i>
            <time datetime="2018-02-21" pubdate>13th June 2018</time>
        </div>
    </div>
</div>

# Objective

In this notebook, we will extract the negation tags and validate the tagged pattern for every sentence. The steps are

* Pattern extraction for every sentence using <a href=https://qa.ird.mu-sigma.com/user/iot/notebooks/DWoW_Diamonds/BMS/code_review/Final_NBs/Pattern%20Extraction.ipynb> Drug, Temporal, Order and Negation pattern extraction notebook</a>
* Get the CoNLL format for every word in the sentence using ctakes output
* Identification of negation words (Cue words) in a sentence
* Detect the scope for every cue word
* Update the existing pattern

In [1]:
import pandas as pd
import numpy as np
import ast
import xmltodict
from cue_scope import load_cue_learner , load_scope_learner
import negtool as negtool
from negtool import feature_extraction, utils
import utils as negutil
from sklearn.externals import joblib
import re

In [21]:
# Checks if a token passed is a drug or not 
def isDrugMention(token):
    with open("C:/Users/varun.behl/Documents/updated_drugs.txt","r") as f:
        drug = f.read()
    try:
        if len(re.findall(r"\b(?=\w)"+token+r"\b(?!\w)",drug, re.IGNORECASE))>=1:
            return 1
    except:
        return 0

## Get CoNLL dictioanary from the CTAKES output

### CoNLL -X format

The ctakes dependency parser returns the output on CoNLL-X format in the xml file. This needs to be converted to a dictionary which can be used to obtain the drug-negation relationship.

The CoNLL-X format definition is as follows

* ID (index in sentence, starting at 1)
* FORM (word form itself)
* LEMMA (word's lemma or stem)
* POS (part of speech)
* CPOSTAG (Coarse-grained part-of-speech tag, where tagset depends on the language)
* FEAT (list of morphological features separated by |)
* HEAD (index of syntactic parent, 0 for ROOT)
* DEPREL (syntactic relationship between HEAD and this word)

In [10]:
# Reads the xml and makes the coNLL dict
def conll_dict_gen(doc2):
    _id = []
    wi =[]
    word = []
    lemma = []
    cpostag = []
    head = []
    deprel = []
    token_dict = {}

    conll_string = '"'
    drug_conll_index = []
    
    # Iterate over every word in the sentence
    for i in range(1,len(doc2['CAS'][ConllDependencyNode])):
        word_conll = doc2['CAS'][ConllDependencyNode][i]
        if word_conll['@id'].encode("utf-8")=='0':
            continue
        
        #Token_dict which has word index and the cpostag
        token_dict[(word_conll['@_id']).encode("utf-8")]=[i,(word_conll['@cpostag']).encode("utf-8")]
        _id.append((word_conll['@_id']).encode("utf-8"))
        wi.append((word_conll['@id']).encode("utf-8"))
        word.append((word_conll['@form']).encode("utf-8"))
        lemma.append((word_conll['@lemma']).encode("utf-8"))
        cpostag.append((word_conll['@cpostag']).encode("utf-8"))
        head.append((word_conll['@_ref_head']).encode("utf-8"))
        deprel.append((word_conll['@deprel']).encode("utf-8"))
        if isDrugMention((word_conll['@form']).encode("utf-8")):
            # This token is a drug
            drug_conll_index.append(int((word_conll['@id']).encode("utf-8"))-1)

    head_index = []
    head_pos = []
    for i in range(0,len(head)):
        try:
            head_index.append(token_dict[str(head[i])][0])
            head_pos.append(token_dict[str(head[i])][1])
        except:
            # Happens when a ref_head is the start of the sentence
            head_index.append(0)
            head_pos.append(cpostag[i])


    coNLL = {}
    for i in range(0 , len(wi)):
        coNLL[i]= {2:str(wi[i]) ,3:str(word[i]),4:str(lemma[i]),5:str(cpostag[i]),'head':str(head_index[i]),'head-pos':str(head_pos[i]),'deprel' :str(deprel[i]) }
    
    return coNLL, drug_conll_index

## Identification of cue words

### Types of negation markers

Affixal negations can be defined as words marked with a negative affix (in English, either the prefixes un-, in-, dis-, a-, an-, non-, im-, il-, ir-, or the suffix -less)

<b>Examples:</b>

(A-)symmetrical, (Dis-)comfort, (Im-)balanced, (In-)consistent, (Non-)stopped, (Un-)eaten, Meaning(-less)

In [47]:
# Detect every word in the document if it is a cue word or not
def predict_cue_words(dev_instances, dev_sentence_dicts):
    dev_fvs = cue_vectorizer.transform(dev_instances).toarray()
    y_pred = cue_ssvm.predict(dev_fvs)
    dev_y_pred = utils.make_complete_labelarray(dev_sentence_dicts, y_pred)
    return dev_y_pred

# Returns the list of cue words
def get_cueWords_list(dev_y_pred,dev_sentence_dicts,sent_count):
    cues = []
    mw_cues = []
    
    # Iterate through every word
    for token_count in range(len(dev_y_pred[sent_count])):
        
        # If the word is a cue word. Check using the predicted y value
        if dev_y_pred[sent_count][token_count]>0:
            
            # Get the type of cue word
            # If the type is affix, then the affix (the prefix or suffix or infix) which makes the word a 
            # a negative word is returned. Else the token word is returned.
            # Func: get_affix_cue - returns the affix if the cue word is a affixal word. Else returns None
            # OUTPUTS:
            # cue_token : The cue word
            # cue_type : 's' - single non affixal cue
            #            'a' - Affixal cue
            #            'm' - multi-word cue
            # cue_pos : The cue word position
            
            affix = utils.get_affix_cue(dev_sentence_dicts[sent_count][token_count][3].lower(), affixal_cue_lexicon)
            cue_type = 's' if affix == None else 'a'
            
            prev_token = None if token_count == 0 else dev_sentence_dicts[sent_count][token_count-1][3].lower()
            cue_token = dev_sentence_dicts[sent_count][token_count][3] if affix == None else affix
            cue_pos = token_count
            
            # Func mwc_start: Returns a value if the current token is part of a multiword cue
            # Append the cue token and cue_pos to multi-word cues list
            if utils.mwc_start(dev_sentence_dicts[sent_count][token_count][3].lower(), prev_token):
                cue_type = 'm'
                mw_cues.append([cue_token, cue_pos])
                
            cues.append([cue_token, cue_pos, cue_type])
        
        # This condition is reached if the number of cue words >0 and there are no single cue word. 
        # Hence checking for multi-word cue occurence
        else:
            cue_token = dev_sentence_dicts[sent_count][token_count][3]
            cue_pos = token_count
            if token_count == 0:
                    prev_token = None 
            else:
                prev_token = dev_sentence_dicts[sent_count][token_count-1][3].lower()

            if utils.mwc_start(dev_sentence_dicts[sent_count][token_count][3].lower(),prev_token):
                mw_cues.append([cue_token, token_count])
            
    return cues, mw_cues


# Predicts the scope of cue word. Whichever value is not equal to 1 is the scope of the negation word
def predict_cueWords_scope(sentence_dicts, dev_instances):
    # Func extract_features_scope: Extracts features for the scope classifier from the sentence dictionaries.
    # OUTPUTS:
    # sentences: (modified) sentence dictionaries
    # dev_instances: a list of feature dictionaries
    # dev_splits: a list of the sentence lengths
    sentences, dev_instances, dev_splits = feature_extraction.extract_features_scope(sentence_dicts, 'prediction')
    dev_fvs = scope_vectorizer.transform(dev_instances).toarray()
    X_dev, y_dev = negutil.make_splits(dev_fvs, [], dev_splits)
    y_pred = scope_ssvm.predict(X_dev)
    return y_pred

### Detect the scope of every cue word

The scope can be obtained by pre-trained scope learner in negtool. 

In [45]:
#Loads cue learner object files

cue_ssvm, cue_vectorizer, cue_lexicon, affixal_cue_lexicon = load_cue_learner()
xmlOut_folderPath = 'C:/Users/varun.behl/Downloads/apache-ctakes-4.0.0/apache-ctakes-4.0.0/testdata_2/output/' 
ConllDependencyNode = 'org.apache.ctakes.typesystem.type.syntax.ConllDependencyNode'

# Loads scope learner object files
scope_ssvm, scope_vectorizer = load_scope_learner()

# Detects the cue-words and finds the scope of each cue word using the xml output from ctakes
def get_negation_scope(key):
    
    xmlFile = 'post'+str(key)+'.txt.xml'
    
    with open(xmlOut_folderPath+xmlFile) as fd:
        doc2 = xmltodict.parse(fd.read())
    
    for i in range(1,len(doc2['CAS'][ConllDependencyNode])):
        if doc2['CAS'][ConllDependencyNode][i]['@id'].encode("utf-8")=='0':
            continue
    
    # dev_sentence_dict is a list of conll dict for every sentence in the given document
    # Mostly there is only one sentence since all documents are tokenized posts.
    coNLL, drug_conll_index = conll_dict_gen(doc2)
    dev_sentence_dicts = []
    dev_sentence_dicts.append(coNLL)
    
    # dev_sd : Sentence dictonaries
    # dev_instances : List of feature dictionaries
    dev_sd, dev_instances = feature_extraction.extract_features_cue(dev_sentence_dicts, cue_lexicon, affixal_cue_lexicon, 'prediction')
    
    # Find if the sentence has a negation word or not
    # If dev_instances is not empty, there are negations
    cue_flag = 1
    numberOfNegation = 0
    if len(dev_instances) == 0 :
        cue_flag = 0
    
    # If negations are present
    n_cues = 0
    if cue_flag:
        # Detect cue words
        dev_y_pred = predict_cue_words(dev_instances, dev_sentence_dicts)
        
        # This is usually only one iteration since all documents are already tokenized posts
        for sent_count in range(len(dev_y_pred)):
            n_cues = 0
            n_cues = sum(i > 0 for i in dev_y_pred[sent_count])
            numberOfNegation = n_cues
            
            # n_mwc: number of multi-word cue words
            # has_mwc: Boolean value. True if there are multi-word cue words in the sentence
            n_mwc, has_mwc = negutil.count_multiword_cues(dev_sentence_dicts[sent_count], dev_y_pred[sent_count])
            if has_mwc:
                n_cues += n_mwc - 1
                
            cues = []
            mw_cues = []
            
            # If there are no cue words in the sentence
            if n_cues == 0:
                dev_sentence_dicts[sent_count]['neg']=False
                dev_sentence_dicts[sent_count]['cues']=[]
                dev_sentence_dicts[sent_count]['mw_cues']=[]
            
            # Get the cue words in the sentence if present    
            else:
                cues, mw_cues =  get_cueWords_list(dev_y_pred, dev_sentence_dicts,sent_count)
                dev_sentence_dicts[sent_count]['neg']=True
                dev_sentence_dicts[sent_count]['cues']=cues
                dev_sentence_dicts[sent_count]['mw_cues']=mw_cues
    
    y_pred = []
    if n_cues != 0:
        try:
            sentence_dicts = dev_sentence_dicts
            y_pred = predict_cueWords_scope(dev_sentence_dicts, dev_instances)
        
        except:
            numberOfNegation = 0
    
    return  y_pred

In [35]:
x = [i for i in get_negation_scope(0)[0] if i != 3 and i !=0]

for a in x:
    get_negation_scope(0)[0].tolist().index(a)
    

2

In [53]:
sentiments = [-1.8, 0, 0, 0, 0, 0, 0, -1.8, 0, 0, 0, 0, 0.0]
N_SCALAR = -0.293

In [146]:
#def valence_updated(key,sentiments):

def get_negated_index(sentiments,N_SCALAR,key):
    indices = []
    for count in range(0,len(get_negation_scope(key))):
        index_counter = 0
        for i in get_negation_scope(key)[count]:
            if i != 3 and i !=1:
                indices.append(index_counter)
            index_counter +=1 
            
    for elm in list(set(indices)):
        sentiments[elm] = sentiments[elm]*N_SCALAR
    return sentiments
   





                   
                   

In [149]:
print len(get_negated_index(sentiments,N_SCALAR))
print len(sentiments)

13
13


In [122]:
sentiments[[0, 1, 3, 4, 5, 7, 8, 10, 11, 12]]

TypeError: list indices must be integers, not list

In [55]:
"Fever is not because of ipilimumab and fatigue is not because of nivolumab"

'Fever is not because of ipilimumab and fatigue is not because of nivolumab'

In [120]:
get_negation_scope(0)

[array([2, 0, 3, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], dtype=int64),
 array([2, 0, 1, 1, 1, 1, 1, 2, 0, 3, 0, 0, 0], dtype=int64)]

### Update the pattern

Check if the identified drug in the sentence falls within the scope of negation word. In that case, the drug is removed from the pattern tag of the sentence

In [7]:
def negation_removal(row_info):
    # Func get_negation_scope : Detects the cue-words and finds the scope of each cue word using the xml output from ctakes
    # OUTPUTS:
    # numberOfNegation : number of cue words
    # drug_conll_index : List of drug indices
    # y_pred_scope : The predicted scope of all the negation words
    key = row_info['Unique_id']
    numberOfNegation, drug_conll_index, y_pred_scope = get_negation_scope(key)
    
    words = ast.literal_eval(row_info['Words'])
    pattern =  ast.literal_eval(row_info['Pattern'])
    index = ast.literal_eval(row_info['Index'])
    
    new_pattern = []
    new_index = []
    new_words = []
    neglist = []
    
    # Iterate through every drug
    for j in range(0,len(drug_conll_index)):
        flag = 0
        
        # Check if the drug is in the scope of any cue word. 
        # For words in the scope of a cue word, the value in y_pred will not be 1
        for neg in range(0,numberOfNegation):
            if y_pred_scope[neg][drug_conll_index[j]] != 1:
                flag += 1
                
        if flag == 0:
            neglist.append(0)
        else:
            neglist.append(1)
            
    drug_count = 0
    
    # Iterate through the pattern in the csv read
    for j in range(0,len(pattern)):
        
        # If the pattern is D, check if it is in the scope of any cue word. If so don't add it to the new pattern
        if pattern[j] == 'D':
            if len(neglist) ==0:
                new_pattern.append(pattern[j])
                new_index.append(index[j])
                new_words.append(words[j])
            elif neglist[drug_count] != 1:
                new_pattern.append(pattern[j])
                new_index.append(index[j])
                new_words.append(words[j])                            
            drug_count+=1
            
        # If the pattern is N, ignore that in the new pattern
        elif pattern[j] == 'N':
            pass
        
        # For any other pattern like Order, add it to the new patten
        else:
            new_pattern.append(pattern[j])
            new_index.append(index[j])
            new_words.append(words[j])  
    return new_pattern , new_index , new_words

In [7]:
Updated_pattern = []
Updated_words = []
Updated_index = []
Unique_id = []

df = pd.read_csv('All_pattern_out_100_user.csv')
for x in range(0,len(df)):
    if(x%100==0):
        print x
    row_info = df.loc[x]
    try:
        pattern , index , words = negation_removal(row_info)
        Updated_pattern.append(pattern)
        Updated_words.append(words)
        Updated_index.append(index)
    except:
        print 'In exception'

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100


In [8]:
# Make the output csv with the updated pattern

Updated_negtool_data = pd.DataFrame(
    {'Updated_pattern': Updated_pattern,
     'Updated_words': Updated_words,
     'Updated_index': Updated_index,
    'Unique_id': df['Unique_id']})

Updated_pattern_df = pd.concat([df,Updated_negtool_data], axis=1, join_axes=[df.index])

In [9]:
Updated_pattern_df.head()

Post_id    User        date  forum  User_id  Sent_id  New_tag  \
0       44  Delray  2016-02-09      3     1084        2        1   
1       44  Delray  2016-02-09      3     1084        4        1   
2       44  Delray  2016-02-09      3     1084        5        1   
3       44  Delray  2016-02-09      3     1084        9        1   
4       44  Delray  2016-02-09      3     1084       12        1   

                         Preprocessed_Tokenized_Sent Unique_id Index  \
0  my husband was diagnosed last may with melanom...      44_2    []   
1  because of the size and depth of the tumor mor...      44_4  [47]   
2                      5 weeks of radiation followed      44_5   [4]   
3  next week he will under go a p e t scan he had...      44_9  [20]   
4  so at this point we wait and see what the p e ...     44_12    []   

       ...               Temp_conversion Temp_index  Temp_pattern  \
0      ...                   ['2015-05']        [4]        ['T0']   
1      ...                            []         []            []   
2      ...                       ['P5W']        [0]        ['T0']   
3      ...       ['2016-W07', '2015-SU']    [0, 14]  ['T0', 'T1']   
4      ...                            []         []            []   

                       Temporal         Words  \
0                  ['last may']            []   
1                            []       ['not']   
2                   ['5 weeks']  ['followed']   
3  ['next week', 'last summer']      ['then']   
4                            []            []   

                                           Drug_date Unique_id Updated_index  \
0             [datetime.datetime(2015, 5, 15, 0, 0)]      44_2            []   
1                                                 []      44_4            []   
2              [datetime.datetime(2016, 1, 5, 0, 0)]      44_5           [4]   
3  [datetime.datetime(2016, 6, 15, 0, 0), datetim...      44_9          [20]   
4                                                 []     44_12            []   

  Updated_pattern Updated_words  
0              []            []  
1              []            []  
2             [O]    [followed]  
3             [O]        [then]  
4              []            []  

[5 rows x 22 columns]

In [10]:
Updated_pattern_df.to_csv('Updated_pattern_out_100_user.csv', index = False)